In [37]:
import konlpy
import json
import urllib.request
import requests
import re

In [149]:
print("보그체로 변환하고 싶은 한국어 텍스트를 입력하시오")
Korean_Text_b = "닭 콩팥 훔친 집사가 남의 집 닭까지 훔치려 한다."
print(Korean_Text_b)

보그체로 변환하고 싶은 한국어 텍스트를 입력하시오
닭 콩팥 훔친 집사가 남의 집 닭까지 훔치려 한다.


In [150]:
Korean_Text = Korean_Text_b.split(sep=' ')
Korean_Text

['닭', '콩팥', '훔친', '집사가', '남의', '집', '닭까지', '훔치려', '한다.']

In [151]:
Toknize_Okt = konlpy.tag.Okt()
Toknize_Hannanum = konlpy.tag.Hannanum()
Toknized_Korean_Okt = []
Toknized_Korean_Hannanum = []


for i in range(len(Korean_Text)):
    Toknized_Korean_Okt.append(Toknize_Okt.pos(Korean_Text[i]))
    Toknized_Korean_Hannanum.append(Toknize_Hannanum.pos(Korean_Text[i]))

    for j in range(len(Toknized_Korean_Okt[i])):
        Toknized_Korean_Okt[i][j] = list(Toknized_Korean_Okt[i][j])

    for j in range(len(Toknized_Korean_Hannanum[i])):
        Toknized_Korean_Hannanum[i][j] = list(Toknized_Korean_Hannanum[i][j])


In [152]:
Toknized_Korean_Okt

[[['닭', 'Noun']],
 [['콩팥', 'Noun']],
 [['훔친', 'Verb']],
 [['집사', 'Noun'], ['가', 'Josa']],
 [['남', 'Noun'], ['의', 'Josa']],
 [['집', 'Noun']],
 [['닭', 'Noun'], ['까지', 'Josa']],
 [['훔치려', 'Verb']],
 [['한다', 'Verb'], ['.', 'Punctuation']]]

In [153]:
Toknized_Korean_Hannanum

[[['닭', 'N']],
 [['콩팥', 'N']],
 [['훔치', 'P'], ['ㄴ', 'E']],
 [['집사', 'N'], ['가', 'J']],
 [['남', 'N'], ['의', 'J']],
 [['집', 'N']],
 [['닭', 'N'], ['까지', 'J']],
 [['훔치', 'P'], ['려', 'E']],
 [['하', 'P'], ['ㄴ다', 'E'], ['.', 'S']]]

In [154]:
Toknized_Korean_Hannanum[0][0][0]

'닭'

In [155]:
def ends_with_jongsung(josa):
    m = re.search(r"[가-힣]?[ㄴ]?",josa)
    if m:
        n = m.group()[-1]
        return (ord(n)-ord("가")) % 28 > 0
    else:
        return None

In [156]:
def josa_change():
    pass

In [157]:
m = re.search(r"[가-힣]?[ㄴ]?",'나ㄴ')
m.group()[-1]

'ㄴ'

In [158]:
valid_grammer_Okt = ['Noun', 'Verb', 'Adjective', 'Determiner', 'Adverb', 'Conjunction', 'Exclamation']
valid_grammer_Hannanum = ['N', 'P', 'M', 'I']


stopwords =['있']


client_id = "DItIw9wyPPPRDPx4Hy46"
client_secret = "L6ZEleX9Mp"
url = "https://openapi.naver.com/v1/papago/n2mt"
url2 = "https://transliterator.herokuapp.com/"
sess = requests.Session()

vogulized_sentence = ''

for word in range(len(Toknized_Korean_Hannanum)):
    for w_divided in range(len(Toknized_Korean_Hannanum[word])):
            if Toknized_Korean_Hannanum[word][w_divided][1] in valid_grammer_Hannanum and Toknized_Korean_Hannanum[word][w_divided][0] not in stopwords:
                print("번역 가능한 품사 단어 : " + Toknized_Korean_Hannanum[word][w_divided][0])
                Toknized_Text_English = urllib.parse.quote(Toknized_Korean_Hannanum[word][w_divided][0])
                data = "source=ko&target=en&text=" + Toknized_Text_English
                request = urllib.request.Request(url)
                request.add_header("X-Naver-Client-Id", client_id)
                request.add_header("X-Naver-Client-Secret", client_secret)
                response = urllib.request.urlopen(request, data=data.encode("utf-8"))
                rescode = response.getcode()

                if rescode==200:
                    response_body = response.read()
                    json_word = json.loads(response_body.decode("utf-8"))
                    result = json_word['message']['result']['translatedText']
                    payload = {'input': result}
                    res = sess.post(url2, data=payload)
                    konglish = res.json()['output'].replace('?','')
                    print(konglish)
                    vogulized_sentence += konglish

                else:
                    print("Error")

            else:
                print("번역 불가능한 품사 단어 : " + Toknized_Korean_Hannanum[word][w_divided][0])
                if Toknized_Korean_Hannanum[word][w_divided][1] == 'E':
                    if Toknized_Korean_Hannanum[word][w_divided][0] == 'ㄴ':
                        Toknized_Korean_Hannanum[word][w_divided][0] = '한'
                    elif Toknized_Korean_Hannanum[word][w_divided][0] == '어':
                        Toknized_Korean_Hannanum[word][w_divided][0] = ''
                    elif Toknized_Korean_Hannanum[word][w_divided][0] == '고' or '게':
                        Toknized_Korean_Hannanum[word][w_divided][0] = '하' + Toknized_Korean_Hannanum[word][w_divided][0]

                    josa_change()

                vogulized_sentence += str(Toknized_Korean_Hannanum[word][w_divided][0])

    vogulized_sentence += ' '

print(vogulized_sentence)
print(Korean_Text_b)

번역 가능한 품사 단어 : 닭
치킨
번역 가능한 품사 단어 : 콩팥
키드니
번역 가능한 품사 단어 : 훔치
스틸링
번역 불가능한 품사 단어 : ㄴ
번역 가능한 품사 단어 : 집사
디컨
번역 불가능한 품사 단어 : 가
번역 가능한 품사 단어 : 남
아더스
번역 불가능한 품사 단어 : 의
번역 가능한 품사 단어 : 집
홈
번역 가능한 품사 단어 : 닭
치킨
번역 불가능한 품사 단어 : 까지
번역 가능한 품사 단어 : 훔치
스틸링
번역 불가능한 품사 단어 : 려
번역 가능한 품사 단어 : 하
하
번역 불가능한 품사 단어 : ㄴ다
번역 불가능한 품사 단어 : .
치킨 키드니 스틸링한 디컨가 아더스의 홈 치킨까지 스틸링하려 하하ㄴ다. 
닭 콩팥 훔친 집사가 남의 집 닭까지 훔치려 한다.
